In [1]:
import csv
import numpy as np
import pandas as pd

list_participant is used to store the number of participant.

In [2]:
csvFile = open("reponse.csv", "r")
reader = csv.reader(csvFile)
list_participant = []
for item in reader:
    if item[0] not in list_participant: 
        list_participant.append(item[0])
csvFile.close()

data frame df_reponse is used to store the data of database table reponse (only with repas, temps and notes)

In [3]:
csvFile = open("reponse.csv", "r")
reader = csv.reader(csvFile)
list1 = []
list2 = []
list3 = []
list4 = []
for item in reader:
    list1.append(item[0])
    list2.append(item[1])
    list3.append(item[3])
    list4.append(item[4])
csvFile.close()
data = {'repas': list2, 'temps': list3, 'notes': list4}
df_reponse = pd.DataFrame(data, index = list1)

data frame df_repas is used to store the data of database table repas (only with repas, id, accompagnment, protein and dessert)

In [4]:
csvFile = open("repas.csv", "r")
reader = csv.reader(csvFile)
list1 = []
list2 = []
list3 = []
list4 = []
list5 = []
for item in reader:
    list1.append(item[0])
    list2.append(item[1])
    list3.append(item[4])
    list4.append(item[5])
    list5.append(item[6])
csvFile.close()
data = {'repas':list1, 'id': list2, 'accompagnment': list3, 'protein': list4, 'dessert': list5}
df_repas = pd.DataFrame(data)

function ddm is defined to calculate the Manhaten distance of two repas, where distance1 refers to distance of repas peu variée, distance2 refers to distance of repas variée

In [5]:
def ddm(series1, series2): 
    distance1 = 0
    distance2 = 0
    for i in range(4,7): 
        distance1 += abs(int(series1[i]) - int(series2[i]))
    for i in range(11,14):
        distance2 += abs(int(series1[i]) - int(series2[i]))
    return distance1, distance2

in this for loop, we first merge dataframe of each participant and df_repas, then calculate two distance.

In [7]:
frames = []
for i in range(0, len(list_participant)-1): 
    df_participant = df_reponse[df_reponse.index == list_participant[i]]
    df_participant1 = pd.merge(df_participant, df_repas, on = 'repas')
    df_participant1.index = df_participant.index
    df_participant2 = pd.concat([df_participant1[0:10].sort_values(by = "temps"), 
                                 df_participant1[10:20].sort_values(by = "temps")], 
                                 axis = 1)
    df_participant2.columns = ['repas_p', 'temps_p', 'notes_p', 
                               'i_p', 'a_p', 'p_p', 'd_p', 
                               'repas_v', 'temps_v', 'notes_v', 
                               'i_v', 'a_v', 'p_v', 'd_v']
    list1 = []
    list2 = []
    for i in range(0,10):
        if i < 9:
            list1.append(ddm(df_participant2.iloc[i], df_participant2.iloc[i+1])[0])
            list2.append(ddm(df_participant2.iloc[i], df_participant2.iloc[i+1])[1])
        else: 
            list1.append(ddm(df_participant2.iloc[i], df_participant2.iloc[0])[0])
            list2.append(ddm(df_participant2.iloc[i], df_participant2.iloc[0])[1])
    df_participant2['score_p'] = list1
    df_participant2['score_v'] = list2
    df_participant3 = df_participant2.ix[:,[3,2,14,10,9,15]]
    frames.append(df_participant3)

H:\anacoda\lib\site-packages\ipykernel_launcher.py:24: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


finally we return a dataframe contains the id, note and distance of two type of repas (i.g. i_p means id peu variée, and i_v means id variée)

In [8]:
result = pd.concat(frames)
result.head()

,i_p,notes_p,score_p,i_v,notes_v,score_v
30,1,5,3,2,4,2
30,7,4,0,6,3,3
30,8,4,1,9,2,2
30,4,4,2,8,2,2
30,9,4,1,5,3,1


all the data stored into result1.csv

In [10]:
result.to_csv('result1.csv')